In [85]:
import pandas as pd

In [86]:
# my spotify historical data

df = pd.read_json('StreamingHistory0.json')

In [87]:
df.head()

,endTime,artistName,trackName,msPlayed
0,2019-08-28 05:38,Ariana Grande,Side To Side,33578
1,2019-08-28 05:39,Alex & Sierra,Gravity,43690
2,2019-08-28 05:40,Pakho Chau,傳聞,11669
3,2019-08-28 05:40,Pakho Chau,一事無成,12373
4,2019-08-28 05:40,Pakho Chau,我的宣言,49898


In [88]:
df['time'] = pd.to_datetime(df.endTime, format='%Y/%m/%d')

In [89]:
df = df[['time', 'artistName', 'trackName', 'msPlayed']]

In [90]:
df.head()

,time,artistName,trackName,msPlayed
0,2019-08-28 05:38:00,Ariana Grande,Side To Side,33578
1,2019-08-28 05:39:00,Alex & Sierra,Gravity,43690
2,2019-08-28 05:40:00,Pakho Chau,傳聞,11669
3,2019-08-28 05:40:00,Pakho Chau,一事無成,12373
4,2019-08-28 05:40:00,Pakho Chau,我的宣言,49898


getting advanced data

In [92]:
# access account by generating Spotify app token

import spotipy.util as util
username = '9z2uftm6g5wisylxrqzc056f0'
client_id ='dc49b39997f14cd792e856d4e4fbb7d3'
client_secret = '1972ad083a954d35bd98b55430a99797'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [93]:
print(token)

BQBmMwJi90R57M-5iN8JX_0X6DBvrI0LPmSSGhI5c9wc40Vj_TR5WsgheV0-Ay_D1viudKwqpgo1OcJdL4GMflHk_FFLTFzLDhqDbT8cK_bflToIFB_ytPJmg1JJtR34ye58YUqJClGeTlbFp3BoM01w8H7z1gKATxDEDYvV_Xs


In [94]:
import requests

def get_id(track_name: str, token: str):
    
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer ' + token,
    }
    params = [
    ('q', track_name),
    ('type', 'track'),
    ]
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 5)
        json = response.json()
        first_result = json['tracks']['items'][0]
        track_id = first_result['id']
        return track_id
    except:
        return None

In [95]:
import spotipy
def get_features(track_id: str, token: str):
    sp = spotipy.Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None

In [96]:
df

,time,artistName,trackName,msPlayed
0,2019-08-28 05:38:00,Ariana Grande,Side To Side,33578
1,2019-08-28 05:39:00,Alex & Sierra,Gravity,43690
2,2019-08-28 05:40:00,Pakho Chau,傳聞,11669
3,2019-08-28 05:40:00,Pakho Chau,一事無成,12373
4,2019-08-28 05:40:00,Pakho Chau,我的宣言,49898
...,...,...,...,...
2656,2020-06-29 19:02:00,1976,方向感,158848
2657,2020-06-29 19:07:00,Quarterback,雨和眼淚,309520
2658,2020-06-29 19:11:00,Bisiugroup,擋一根,236133
2659,2020-06-29 19:16:00,S.H.E,你曾是少年,260280


In [106]:
# create new df
feature = pd.DataFrame(columns=['time', 'artistName', 'trackName', 'msPlayed','danceability', 'energy', 'key', 
                                'loudness', 'mode', 'speechiness', 'acousticness', 
                                'instrumentalness', 'liveness', 'valence', 'tempo', 
                                'type', 'duration_ms'])


for i in range(df.shape[0]):
    track_id = get_id(df.iloc[i, 2], token)
    info = get_features(track_id, token)
    if info:
        feature = feature.append({'time':df.iloc[i,0], 'artistName':df.iloc[i,1],
                                  'trackName': df.iloc[i, 2], 'msPlayed':df.iloc[i, 3],
                                  'danceability':info['danceability'],
                                  'energy':info['energy'],
                                  'key':info['key'],
                                  'loudness':info['loudness'], 
                                  'mode':info['mode'], 
                                  'speechiness':info['speechiness'], 
                                  'acousticness':info['acousticness'],
                                  'instrumentalness':info['instrumentalness'], 
                                  'liveness':info['liveness'], 
                                  'valence':info['valence'], 
                                  'tempo':info['tempo'], 
                                  'type':info['type'], 
                                  'duration_ms':info['duration_ms']},ignore_index=True)
    else:
        pass

In [107]:
feature

,time,artistName,trackName,msPlayed,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms
0,2019-08-28 05:38:00,Ariana Grande,Side To Side,33578,0.650,0.736,6,-5.840,0,0.2290,0.05280,0,0.2350,0.6130,159.173,audio_features,226160
1,2019-08-28 05:39:00,Alex & Sierra,Gravity,43690,0.764,0.326,7,-9.883,1,0.0342,0.73100,0.0013,0.2400,0.2370,123.980,audio_features,245827
2,2019-08-28 05:40:00,Pakho Chau,傳聞,11669,0.599,0.408,10,-6.649,1,0.0273,0.53900,0,0.0998,0.4230,96.161,audio_features,247018
3,2019-08-28 05:40:00,Pakho Chau,一事無成,12373,0.907,0.662,1,-3.627,0,0.0554,0.19000,0,0.4680,0.8470,101.036,audio_features,192787
4,2019-08-28 05:40:00,Pakho Chau,我的宣言,49898,0.468,0.433,5,-8.039,1,0.0245,0.63900,0,0.1310,0.4160,84.034,audio_features,277507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2619,2020-06-29 19:02:00,1976,方向感,158848,0.493,0.483,4,-8.262,1,0.0294,0.08660,0.000704,0.0868,0.0526,104.981,audio_features,357720
2620,2020-06-29 19:07:00,Quarterback,雨和眼淚,309520,0.273,0.602,3,-6.967,1,0.0340,0.00387,1.95e-06,0.1260,0.1420,166.019,audio_features,309520
2621,2020-06-29 19:11:00,Bisiugroup,擋一根,236133,0.518,0.800,6,-6.412,0,0.0452,0.07990,0,0.0980,0.4300,76.064,audio_features,236133
2622,2020-06-29 19:16:00,S.H.E,你曾是少年,260280,0.530,0.603,2,-7.058,1,0.0309,0.11600,0,0.0925,0.3240,155.982,audio_features,266651


In [108]:
feature.to_csv('my_music.csv')